# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 17 2023 11:54AM,261203,10,Enova-11667,Emerginnova,Released
1,Apr 17 2023 11:49AM,261202,19,9387660,"Innogenix, LLC",Released
2,Apr 17 2023 11:49AM,261202,19,9387661,"Innogenix, LLC",Released
3,Apr 17 2023 11:49AM,261202,19,9387662,"Innogenix, LLC",Released
4,Apr 17 2023 11:49AM,261202,19,9387663,"Innogenix, LLC",Released
5,Apr 17 2023 11:49AM,261202,19,9387664,"Innogenix, LLC",Released
6,Apr 17 2023 11:49AM,261202,19,9387665,"Innogenix, LLC",Released
7,Apr 17 2023 11:49AM,261202,19,9387666,"Innogenix, LLC",Released
8,Apr 17 2023 11:49AM,261202,19,9387667,"Innogenix, LLC",Released
9,Apr 17 2023 11:49AM,261202,19,9387670,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,261195,Released,1
23,261196,Released,1
24,261201,Released,29
25,261202,Released,20
26,261203,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261195,NaN,NaN,1.0
261196,NaN,NaN,1.0
261201,NaN,NaN,29.0
261202,NaN,NaN,20.0
261203,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261145,10.0,2.0,2.0
261146,0.0,0.0,1.0
261147,0.0,0.0,1.0
261160,0.0,0.0,46.0
261161,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261145,10,2,2
261146,0,0,1
261147,0,0,1
261160,0,0,46
261161,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261145,10,2,2
1,261146,0,0,1
2,261147,0,0,1
3,261160,0,0,46
4,261161,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261145,10,2,2
1,261146,,,1
2,261147,,,1
3,261160,,,46
4,261161,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 17 2023 11:54AM,261203,10,Emerginnova
1,Apr 17 2023 11:49AM,261202,19,"Innogenix, LLC"
21,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.
50,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc"
51,Apr 17 2023 11:16AM,261194,16,Sartorius Stedim Filters Inc.
52,Apr 17 2023 11:16AM,261195,10,"ClearSpec, LLC"
53,Apr 17 2023 11:14AM,261193,19,"NAPP Technologies, LLC"
54,Apr 17 2023 11:09AM,261190,10,"Methapharm, Inc."
55,Apr 17 2023 11:04AM,261183,10,Bio-PRF
101,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 17 2023 11:54AM,261203,10,Emerginnova,,,1
1,Apr 17 2023 11:49AM,261202,19,"Innogenix, LLC",,,20
2,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.,,,29
3,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc",,,1
4,Apr 17 2023 11:16AM,261194,16,Sartorius Stedim Filters Inc.,,,1
5,Apr 17 2023 11:16AM,261195,10,"ClearSpec, LLC",,,1
6,Apr 17 2023 11:14AM,261193,19,"NAPP Technologies, LLC",,,1
7,Apr 17 2023 11:09AM,261190,10,"Methapharm, Inc.",,,1
8,Apr 17 2023 11:04AM,261183,10,Bio-PRF,,,46
9,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:54AM,261203,10,Emerginnova,1,,
1,Apr 17 2023 11:49AM,261202,19,"Innogenix, LLC",20,,
2,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.,29,,
3,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc",1,,
4,Apr 17 2023 11:16AM,261194,16,Sartorius Stedim Filters Inc.,1,,
5,Apr 17 2023 11:16AM,261195,10,"ClearSpec, LLC",1,,
6,Apr 17 2023 11:14AM,261193,19,"NAPP Technologies, LLC",1,,
7,Apr 17 2023 11:09AM,261190,10,"Methapharm, Inc.",1,,
8,Apr 17 2023 11:04AM,261183,10,Bio-PRF,46,,
9,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:54AM,261203,10,Emerginnova,1,,
1,Apr 17 2023 11:49AM,261202,19,"Innogenix, LLC",20,,
2,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.,29,,
3,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc",1,,
4,Apr 17 2023 11:16AM,261194,16,Sartorius Stedim Filters Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:54AM,261203,10,Emerginnova,1,NaN,NaN
1,Apr 17 2023 11:49AM,261202,19,"Innogenix, LLC",20,NaN,NaN
2,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.,29,NaN,NaN
3,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc",1,NaN,NaN
4,Apr 17 2023 11:16AM,261194,16,Sartorius Stedim Filters Inc.,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:54AM,261203,10,Emerginnova,1,0.0,0.0
1,Apr 17 2023 11:49AM,261202,19,"Innogenix, LLC",20,0.0,0.0
2,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.,29,0.0,0.0
3,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc",1,0.0,0.0
4,Apr 17 2023 11:16AM,261194,16,Sartorius Stedim Filters Inc.,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3134230,176,4.0,0.0
15,261169,3,18.0,0.0
16,261194,1,0.0,0.0
18,261161,1,0.0,0.0
19,1828184,27,2.0,10.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3134230,176,4.0,0.0
1,15,261169,3,18.0,0.0
2,16,261194,1,0.0,0.0
3,18,261161,1,0.0,0.0
4,19,1828184,27,2.0,10.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,176,4.0,0.0
1,15,3,18.0,0.0
2,16,1,0.0,0.0
3,18,1,0.0,0.0
4,19,27,2.0,10.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,176.0
1,15,Released,3.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19
Status,,,,,
Completed,0.0,0.0,0.0,0.0,10.0
Executing,4.0,18.0,0.0,0.0,2.0
Released,176.0,3.0,1.0,1.0,27.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19
0,Completed,0.0,0.0,0.0,0.0,10.0
1,Executing,4.0,18.0,0.0,0.0,2.0
2,Released,176.0,3.0,1.0,1.0,27.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19
0,Completed,0.0,0.0,0.0,0.0,10.0
1,Executing,4.0,18.0,0.0,0.0,2.0
2,Released,176.0,3.0,1.0,1.0,27.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()